In [ ]:
import cv2
import numpy as np
import rootutils
import mediapy

#  auto-reloads all modules
%load_ext autoreload
%autoreload 2 

# import python modules relatively to the project root directory
root = rootutils.setup_root(".", indicator="homeworks", pythonpath=True)

# data directory
DATA_DIR = root / "data"

### **Преобразования из мировой системы координат в координаты на изображениях**

<center>
    <figure>
        <img src="notebooks/images/09/twoview.jpg" width=30%"/>
    </figure>
</center>

- Мы рассматривали преобразования из мировой системы координат в систему координат изображения камеры:
  $$
    \lambda\cdot\widetilde{\mathbf{x}} = \mathbf{K}[\mathbf{R} \mid \mathbf{t}] \cdot \widetilde{\mathbf{w}}
  $$
  $$
    \lambda' \cdot \widetilde{\mathbf{x}}' = \mathbf{K}' [\mathbf{R}' \mid \mathbf{t}'] \cdot \widetilde{\mathbf{w}}
  $$

- Рассмотрим случаи, когда можно установить связь между координатами на изображениях напрямую, без использования мировой системы координат.

### **Гомография (проективное преобразование)**

<center>
    <figure>
        <img src="images/09/matches.jpg" width=50%"/>
    </figure>
</center>

Если **прямые** на одном изображении отображаются в **прямые**
на другом изображении, то координаты **соответствующих точек** на изображениях 
преобразуются с помощью гомографии:

$$
    \lambda\cdot
    \begin{bmatrix}
        x' \\
        y' \\
        1
    \end{bmatrix} =
    \begin{bmatrix}
        h_{11} & h_{12} & h_{13} \\
        h_{21} & h_{22} & h_{23} \\
        h_{31} & h_{32} & h_{33}
    \end{bmatrix}
    \cdot  
    \begin{bmatrix}
        x \\
        y \\
        1
    \end{bmatrix}
$$
$$
    \lambda\cdot\widetilde{\mathbf{x}}' = \mathbf{H} \cdot \widetilde{\mathbf{x}}
$$

**Свойства гомографии:**

- При гомографии прямые линии отображаются в прямые. 
  При этом углы и масштаб меняются.

- Матрицу гомографии можно умножать на множитель - гомография не измениться.

- В однородных координатах гомография - линейное преобразование, 
  но в декартовых координатах оно нелинейно:
    $$
        x' = \frac{h_{11} x + h_{12} y + h_{13}}{h_{31} x + h_{32} y + h_{33}} 
    $$
    $$
        y' = \frac{h_{21} x + h_{22} y + h_{23}}{h_{31} x + h_{32} y + h_{33}} 
    $$

    Обозначение гомографии в декартовых координатах:
    $$
        \mathbf{x}'=\mathbf{Hom}[\mathbf{x}]
    $$

- Матрица гомографии $\mathbf{H}$ имеет 8 степеней свободы 
(ее можно умножить на произвольное число, не меняя 
результат преобразования). 

- Для вычисления гомографии
необходимо как минимум 4 пары соответствующих точек 
(каждая пара дает 2 уравнения, всего нужно 8 уравнений).

### **Гомография в случае плоской сцены**

<center>
    <figure>
        <img src="images/09/homography2.jpg" width=50%"/>
    </figure>
</center>

- Если сцена плоская, то прямые на одном изображении соответствуют прямым на другом изображении. 

- Значит координаты точек на изображениях связаны гомографией:
    $$
        \lambda\cdot\widetilde{\mathbf{x}}' = \mathbf{H} \cdot \widetilde{\mathbf{x}}
    $$

#### **Гомография при вращении камеры вокруг оптического центра**
<center>
    <figure>
        <img src="images/09/homography1.jpg" width=40%"/>
    </figure>
</center>

- Возьмем два изображения, полученные при вращении камеры вокруг оптического центра. 

- Хотя сцена может быть объемной, координаты соответствующих точек на изображениях связаны гомографией.

- Если мировая система координат совпадает с системой координат камеры в  первом положении, то для первой камеры:
    $$
        \lambda\cdot\widetilde{\mathbf{x}} = \mathbf{K} \cdot 
        \left[
            \mathbf{I} \mid \mathbf{0}
        \right] \cdot 
        \widehat{\mathbf{w}}
    $$

    Для второй камеры:
    $$
        \lambda'\cdot\widetilde{\mathbf{x}}' = \mathbf{K}' \cdot 
        \left[
            \mathbf{R} \mid \mathbf{0}
        \right] \cdot 
        \widehat{\mathbf{w}}
    $$

    Cледовательно:
    $$
        \lambda'\cdot\widetilde{\mathbf{x}}' = \mathbf{K}' \cdot \mathbf{R} \cdot \mathbf{K}^{-1} \cdot \lambda\cdot\widetilde{\mathbf{x}}
    $$  

- Получаем гомографию:
    $$
        \mathbf{H} = \mathbf{K}' \cdot \mathbf{R} \cdot \mathbf{K}^{-1}
    $$
    $$
        \widetilde{\mathbf{x}}' = \mathbf{H} \cdot \widetilde{\mathbf{x}}
    $$

**Подготовка данных плоской сцены**

In [ ]:
# загрузка видео с шахматной доской
video_file = DATA_DIR / "chessboard.avi"
video = cv2.VideoCapture(video_file)

# ограничение на количество обрабатываемых кадров
frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
num_frames = min(frame_count, 500)
print(f"Total frames in video: {frame_count}, processing: {num_frames}")

# число углов шахматной доски по горизонтали и вертикали
board_pattern = (10, 7)

# списки для хранения кадров и координат углов шахматной доски
frames = []
points = []

for _ in range(num_frames):
    valid, frame = video.read()

    if not valid:
        break
    
    image = frame.copy()
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # поиск углов шахматной доски
    complete, pts = cv2.findChessboardCorners(gray, board_pattern)

    # сохранение кадров и координат углов шахматной доски
    if complete:
        frames.append(frame)
        points.append(pts[:, 0, :])

        # отрисовка найденных углов
        cv2.drawChessboardCorners(image, board_pattern, pts, complete)
    
    cv2.imshow("Chessboard", image)
    key = cv2.waitKey(10)

    if key == ord(" "):     # Space: Pause
        key = cv2.waitKey()
    if key == ord("q"):     # Q: Exit
        break

cv2.destroyAllWindows()
video.release()

# преобразование списков в numpy массивы
frames = np.array(frames).astype(np.uint8)
points = np.array(points).astype(np.float32)

In [ ]:
# вывод размеров полученных данных
print(f"frames.shape = {frames.shape}")
print(f"points.shape = {points.shape}")

**Отобразим найденные углы шахматной доски на двух кадрах**

In [ ]:
# данные первого и последнего кадров
frame1 = frames[0].copy()
points1 = points[0]

frame2 = frames[-1].copy()
points2 = points[-1]

# отрисовка найденных углов на кадрах
for (x1, y1), (x2, y2) in zip(points1, points2):
    cv2.circle(frame1, (int(x1), int(y1)), 7, (255, 0, 255), -1)
    cv2.circle(frame2, (int(x2), int(y2)), 7, (255, 0, 255), -1)

# отображение результатов
mediapy.show_images({
        "Image 1": frame1[:, :, ::-1],
        "Image 2": frame2[:, :, ::-1]
    }, 
    border=True, width=400, columns=2
)

### **Вычисление гомографии**

<center>
    <figure>
        <img src="images/09/hom_est.jpg" width=70%"/>
    </figure>
</center>

**Обозначения:**

- $i=1,2,\ldots,N$ - номер точки, всего $N$ точек

- координаты точек на первом изображении
    $$
    \mathbf{x}_i = 
    \begin{bmatrix}
        x_i \\
        y_i
    \end{bmatrix}
    $$

-  координаты соответствующих точек на втором изображении
    $$
        \mathbf{x}'_i = 
        \begin{bmatrix}
            x'_i \\
            y'_i
        \end{bmatrix}
    $$

- преобразование гомографии точек с первого изображения на второе
    $$
        \widehat{\mathbf{x}}'_i = 
        \begin{bmatrix}
            \widehat{x}'_i \\
            \widehat{y}'_i
        \end{bmatrix}=
        \mathbf{Hom}[\mathbf{x}_i]
    $$

**Вычисление гомографии.**

Матрицу гомографии можно вычислить с помощью метода максимального правдоподобия:
$$
    \widehat{\mathbf{H}} = \argmin_{\mathbf{H}} \left(-\sum_{i} 
    \log\left[\mathrm{Norm}_{\mathbf{x}'_i}\left[\widehat{\mathbf{x}}'_i\,,\sigma^2\mathbf{I}\right]\right]
    \right)
$$

- Можно упростить это выражение и свести к задаче наименьших квадратов:
$$
    \widehat{\mathbf{H}} = \argmin_{\mathbf{H}} 
    \frac{1}{N}\sum_{i} 
    \left\|\mathbf{x}'_i - \widehat{\mathbf{x}}'_i]\right\|^2
$$

- Величина 
    $$
        \left\|\mathbf{x}'_i - \widehat{\mathbf{x}}_i\right\|^2=
        \left(x'_i-\frac{h_{11}x_i + h_{12}y_i + h_{13}}{h_{31}x_i + h_{32}y_i + h_{33}}\right)^2 +
            \left(y'_i-\frac{h_{21}x_i + h_{22}y_i + h_{23}}{h_{31}x_i + h_{32}y_i + h_{33}}\right)^2
    $$
    называется ошибкой проекции для точки $i$ или геометрической ошибкой.

- Средняя ошибка проекции по всем точкам показывает, насколько хорошо гомография описывает преобразование между изображениями:
$$
    \mathrm{ProjError} = \frac{1}{N}\sum_{i} 
    \left\|\mathbf{x}'_i - \widehat{\mathbf{x}}_i\right\|^2
$$

- К сожалению, это выражение нелинейно по параметрам гомографии. Поэтому для решения задачи наименьших квадратов обычно используют численные методы оптимизации (например, метод Левенберга-Марквардта).

- Для вычисления начальной оценки гомографии используется метод DLT (Direct Linear Transform).    

**Пример вычисления гомографии с помощью OpenCV**

In [ ]:
# вычисление гомографии с помощью OpenCV
cv_H, inlier_mask = cv2.findHomography(points1, points2)

with np.printoptions(precision=4, suppress=True):
    print("OpenCV Homography:\n", cv_H)

print(f"\nInliers count: {np.sum(inlier_mask)} / {len(inlier_mask)}")

**Оценим качество гомографии, вычислив среднюю ошибку проекции.**

In [ ]:
# преобразование из декартовых координат в однородные
hom_points = np.hstack((points1, np.ones((len(points1), 1))))

# преобразование точек с помощью гомографии
hat_points = (cv_H @ hom_points.T).T

# преобразование обратно в декартовы координаты
hat_points = hat_points[:, :2] / hat_points[:, 2:3]

# отображение результатов преобразования
image1 = frame1.copy()
image2 = frame2.copy()

for (x1, y1), (x2, y2), (hx, hy) in zip(points1, points2, hat_points):
    cv2.circle(image1, (int(x1), int(y1)), 4, (255, 0, 255), -1)
    cv2.circle(image2, (int(x2), int(y2)), 7, (255, 0, 255), -1) 
    cv2.circle(image2, (int(hx), int(hy)), 4, (0, 255, 255), -1)

mediapy.show_images({
        "Image 1": image1[:, :, ::-1],
        "Image 2": image2[:, :, ::-1]
    }, 
    border=False, width=800
)

In [ ]:
# вычисление средней ошибки проекции
proj_error = np.linalg.norm(points2 - hat_points, axis=1).mean()

print(f"Mean Projection Error (OpenCV): {proj_error:.4f} pixels")

### **Вычисление гомографии с помощью метода DLT**

- Предполагаем, что ошибок проекции нет:
    $$
        \mathbf{x}'_i =
        \widehat{\mathbf{x}}'_i
    $$
    $$
            \lambda_i\cdot
            \begin{bmatrix}
                x'_i \\ y'_i \\ 1
            \end{bmatrix} =
            \begin{bmatrix}
                h_{11} & h_{12} & h_{13} \\
                h_{21} & h_{22} & h_{23} \\
                h_{31} & h_{32} & h_{33}
            \end{bmatrix}\cdot
            \begin{bmatrix}
                x_i \\ y_i \\ 1
            \end{bmatrix}
    $$

- Умножим эту формулу слева векторно на 
    $$
    \begin{bmatrix}
        x'_i \\ y'_i \\ 1
    \end{bmatrix}\times
    $$
    Получим:
    $$
        \mathbf{0} =
        \begin{bmatrix}
            x'_i \\ y'_i \\ 1
        \end{bmatrix} \times
        \begin{bmatrix}
            h_{11} & h_{12} & h_{13} \\
            h_{21} & h_{22} & h_{23} \\
            h_{31} & h_{32} & h_{33}
        \end{bmatrix}\cdot
        \begin{bmatrix}
            x_i \\ y_i \\ 1
        \end{bmatrix}
    $$

- Раскроем векторное произведение и получим систему линейных уравнений:
$$
    \begin{bmatrix}
        0 & -1 & y'_i \\
        1 & 0 & -x'_i \\
        -y'_i & x'_i & 0
    \end{bmatrix} \cdot
    \begin{bmatrix}
        h_{11} & h_{12} & h_{13} \\
        h_{21} & h_{22} & h_{23} \\
        h_{31} & h_{32} & h_{33}
    \end{bmatrix} \cdot
    \begin{bmatrix}
        x_i \\ y_i \\ 1
    \end{bmatrix} = \mathbf{0}
$$

- Два из этих трех уравнений линейно независимы. Возьмем первые два уравнения:
    $$
        \begin{bmatrix}
            0 & 0 & 0 & -x_i & -y_i & -1 & y'_i x_i & y'_i y_i & y'_i \\
            x_i & y_i & 1 & 0 & 0 & 0 & -x'_i x_i & -x'_i y_i & -x'_i
        \end{bmatrix}
        \cdot\boldsymbol{\theta}=\mathbf{0}
    $$
    где
    $$
        \boldsymbol{\theta} =
        [h_{11}\,, h_{12}\,, h_{13}\,, h_{21}\,, h_{22}\,, h_{23}\,, h_{31}\,, h_{32}\,, h_{33}]^\top
    $$

- Для $N$ соответствующих точек получаем систему уравнений:
    $$
        \mathbf{\Omega}\cdot\boldsymbol{\theta}=\mathbf{0}
    $$
    где матрица $\mathbf{\Omega}$ размера $2N\times 9$ составляется из блоков:
    $$
        \begin{bmatrix}
            0 & 0 & 0 & -x_i & -y_i & -1 & y'_i x_i & y'_i y_i & y'_i \\
            x_i & y_i & 1 & 0 & 0 & 0 & -x'_i x_i & -x'_i y_i & -x'_i
        \end{bmatrix}
    $$

- Оценка для $\boldsymbol{\theta}$ находится как решение задачи:
    $$
        \widehat{\boldsymbol{\theta}} = 
        \argmin_{\boldsymbol{\theta}} 
        \left\|{\mathbf{\Omega}\cdot\boldsymbol{\theta}} \right\|^2
        \quad \text{при условии} \quad \left\|{\boldsymbol{\theta}}\right\|=1
    $$

-  Такое решение находится как правый столбец матрицы $\mathbf{V}$ в SVD-разложении
    $$
        \mathbf{\Omega} = \mathbf{U}\mathbf{L}\mathbf{V}^\top
    $$

-  Матрица гомографии $\mathbf{H}$ получается из вектора $\widehat{\boldsymbol{\theta}}$ перестановкой его элементов в матрицу $3\times 3$.

- Поскольку гомография определяется с точностью до множителя, обычно нормируют матрицу гомографии так, чтобы $h_{33}=1$.

#### **Задача 1: Вычисление гомографии с помощью метода DLT**

Используя метод DLT, реализуйте функцию для вычисления матрицы гомографии по заданным соответствующим точкам.

```python
    def compute_homography_dlt(points1, points2):
        """
        Вычисляет матрицу гомографии между двумя наборами соответствующих точек.
        Returns:
            Матрица гомографии размером (3, 3).
        """
        ...
        return H
```

In [ ]:
from homeworks.homework_09 import compute_homography_dlt

# Вычисление гомографии с помощью DLT
dlt_H = compute_homography_dlt(points1, points2)

with np.printoptions(precision=4, suppress=True):
    print("DLT Homography:\n", dlt_H)

# Оценка качества гомографии, найденной с помощью DLT

# преобразование точек с помощью гомографии
hom_points = np.hstack((points1, np.ones((len(points1), 1))))
hat_points = (dlt_H @ hom_points.T).T
hat_points = hat_points[:, :2] / hat_points[:, 2:3]

# вычисление средней ошибки проекции
proj_error = np.linalg.norm(points2 - hat_points, axis=1).mean()

print(f"\nMean Projection Error (DLT): {proj_error:.4f} pixels")

### **Применение гомографии для преобразования изображений**

Вычислив гомографию можно использовать ее для преобразования изображений с помощью функции `cv2.warpPerspective()`. 

In [ ]:
image1 = frames[0].copy()
image2 = frames[-1].copy()

height, width = image2.shape[:2]

# преобразование первого кадра с помощью гомографии OpenCV
warped_image = cv2.warpPerspective(
    image1,          # исходное изображение
    cv_H,            # матрица гомографии
    (width, height), # размер выходного изображения
    cv2.INTER_LINEAR # линейная интерполяция
)

# создание смешанного изображения
blended_image = cv2.addWeighted(warped_image, 0.5, image2, 0.5, 0)

# визуализация результатов
mediapy.show_images({
        "Image 1": image1[:, :, ::-1],
        "Image 2": image2[:, :, ::-1],
        "Warped Image": warped_image[:, :, ::-1],
        "Blended Image": blended_image[:, :, ::-1]
    }, 
    columns=2, width=500, border=False
)

#### **Построение панорамного изображения**

<center>
    <figure>
        <img src="images/09/panorama.jpg" width=70%"/>
    </figure>
</center>

- Три изображения сцены, для которых камера поворачивается вокруг
своего оптического центра.

- Для каждого изображения построены пять соответствующих ключевых
точек.

- Панорама строится при помощи гомографий, преобразующих первое и
третье изображения в систему координат второго изображения.

**Подготовка данных**

Возьмем два изображения, полученные при вращении камеры вокруг оптического центра. 

In [ ]:
image1 = cv2.imread(DATA_DIR / "hill01.jpg")
image2 = cv2.imread(DATA_DIR / "hill02.jpg")

mediapy.show_images({
        "Image 1": image1[:, :, ::-1],
        "Image 2": image2[:, :, ::-1]
    }, 
    border=False, width=500, columns=2
)

**Вычислим SIFT ключевые точки и дескрипторы.**

In [ ]:
# создание SIFT детектора с заданными параметрами
sift = cv2.SIFT_create(
    nfeatures = 1000,
    edgeThreshold = 6,
    contrastThreshold = 0.04,
    nOctaveLayers = 3,
    sigma = 1.6
)
    
# детектирование ключевых точек и вычисление дескрипторов
keypoints1, descriptors1 = sift.detectAndCompute(image1, None)
keypoints2, descriptors2 = sift.detectAndCompute(image2, None)

# визуализация ключевых точек на изображениях
out_image1 = image1.copy()
out_image2 = image2.copy()

cv2.drawKeypoints(image1, keypoints1, out_image1)
cv2.drawKeypoints(image2, keypoints2, out_image2)

# отображение результатов
mediapy.show_images({
    "image1": out_image1[:, :, ::-1], 
    "image2": out_image2[:, :, ::-1]}, 
    border=True, width=400
)

**Сопоставим ключевые точки между двумя изображениями с помощью Cross-Check Matching.**

In [ ]:
# Brute-Force Matcher с Cross-Check
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)

# Cross-Check Matching
matches = bf.match(descriptors1, descriptors2)

# Draw matches.
matches_image = cv2.drawMatches(image1,
    keypoints1,
    image2,
    keypoints2,
    matches,
    outImg=None,
)

mediapy.show_image(matches_image[:, :, ::-1], width=800)

#### **Алгоритм RANSAC для оценки гомографии**
- Пары соответствующих точек состоят из inliers и outliers.

- Перед оценкой параметров гомографии необходимо отфильтровать outliers.

- C помощью RANSAC можно отфильтровать outliers и оценить параметры гомографии только на inliers.

**RANSAC**

- повторить $N$ раз
    - выбрать 4 случайных соответствующих точек $\mathbf{x}_i\leftrightarrow\mathbf{x}'_i$ и вычислить $\mathbf{H}$ методом DLT
    
    -  разделить точки на inliers и outliers по порогу $T$ и ошибке проекции:
        $$
            \|\mathbf{x}'_i - \widehat{\mathbf{x}}'_i\|<T
        $$

-  выбрать $\mathbf{H}$ c наибольшим числом inliers в качестве начальной оценки алгоритма Левенберга-Марквардта

-  методом Левенберга-Марквардта вычислить финальную гомографию $\mathbf{H}$ только на inliers

**Найдем гомографию с помощью RANSAC алгоритма**

In [ ]:
# преобразуем из списка матчей keypoints в массивы точек
points1, points2 = [], []

for i in range(len(matches)):
    m = matches[i]
    points1.append(keypoints1[m.queryIdx].pt)
    points2.append(keypoints2[m.trainIdx].pt)

points1 = np.array(points1, dtype=np.float32)
points2 = np.array(points2, dtype=np.float32)

# одобнее найти гомографию с помощью RANSAC
# из image2 в image1
H, inlier_mask = cv2.findHomography(points2, points1, cv2.RANSAC)

# преобразуем image2  с помощью найденной гомографии
height, width = image1.shape[:2]
warped_image = cv2.warpPerspective(image2, H, (width * 2, height))

# скопируем image1 в левую часть warped_image
warped_image[:, :width] = image1 

# визуализация результата
mediapy.show_image(warped_image[:, :, ::-1], width=800)

#### **Рассмотрим задачу стабилизации видео с помощью гомографии**

In [ ]:
# загрузка и воспроизведение видео с помощью mediapy
frames = mediapy.read_video(DATA_DIR / "traffic.avi")
mediapy.show_video(frames, title = "Traffic", fps=20, width=600)

- Поскольку сцена почти плоская, преобразование между кадрами можно аппроксимировать гомографией.

- Идея стабилизации видео состоит в том, чтобы каждый кадр преобразовать с помощью гомографии к первому кадру.

- Ключевые точки будем вычислять с помощью функции ``cv2.goodFeaturesToTrack()`` (реализация Shi-Tomasi алгоритма детектора ключевых точек).

- Сопоставление ключевых точек между кадрами будем выполнять с помощью оптического потока Лукаса-Канаде.

Преобразуем, например, 100-й кадр в первый:


In [ ]:
image1 = frames[0].copy()
image2 = frames[100].copy()

gray1 = cv2.cvtColor(image1, cv2.COLOR_RGB2GRAY)
gray2 = cv2.cvtColor(image2, cv2.COLOR_RGB2GRAY)

# вычисление ключевых точек с помощью Shi-Tomasi метода
points1 = cv2.goodFeaturesToTrack(gray1, 2000, 0.01, 10)
points1 = points1[:, 0, :]

# с помощью оптического потока Лукаса-Канаде
# точки points1 сопоставляются с points2
points2, _, _ = cv2.calcOpticalFlowPyrLK(gray1, gray2, points1, None)

# отрисовка найденных точек на кадрах
for (x1, y1), (x2, y2) in zip(points1, points2):
    cv2.circle(image1, (int(x1), int(y1)), 2, (255, 0, 255), -1)
    cv2.circle(image2, (int(x2), int(y2)), 2, (255, 0, 255), -1)
    
# отображение результатов
mediapy.show_images({
        "Image 1": image1,
        "Image 2": image2
    }, 
    border=True, width=500, columns=2
)

In [ ]:
# вычисление гомографии из image2 в image1
H, _ = cv2.findHomography(points2, points1, cv2.RANSAC)

image1 = frames[0].copy()
image2 = frames[100].copy()

# применение гомографии для преобразования image2 в систему координат image1
height, width = image1.shape[:2]
warped_image = cv2.warpPerspective(image2, H, (width, height))

# отображение результатов
mediapy.show_images({
        "Image 1": image1,
        "Image 2": image2,
        "Warped Image 2": warped_image
    }, 
    border=True, width=500, columns=3
)

#### **Задача 2: Стабилизация видео с помощью гомографии**

Напишите функцию для стабилизации видео с помощью гомографии между кадрами.

```python
def stabilize_video_with_homography(frames):
    """
    Стабилизирует видео с помощью гомографии между кадрами.
    Args:
        frames: numpy массив кадров
    Returns:
        Стабилизированные кадры видео (numpy массив).
    """
    ...
    
    assert frames.shape == stabilized_frames.shape

    return stabilized_frames
```

Протестируем функцию ``stabilize_video_with_homography()``

In [ ]:
from homeworks.homework_09 import stabilize_video_with_homography

stabilized_frames = stabilize_video_with_homography(frames)

# конкатенируем фреймы вдоль горизонтали
out_frames = np.concatenate((frames, stabilized_frames), axis=1)

# отобразим результат стабилизации
mediapy.show_video(out_frames, title = "Video vs Stabilized Video", fps=20, width=600)

#### **Частные случаи гомографии**

<center>
    <figure>
        <img src="images/09/hom1.jpg" width=40%"/>
        <img src="images/09/hom2.jpg" width=30%"/>
    </figure>
</center>

$$
    \lambda\cdot
    \begin{bmatrix}
        x' \\
        y' \\
        1
    \end{bmatrix} =
    \begin{bmatrix}
        h_{11} & h_{12} & h_{13} \\
        h_{21} & h_{22} & h_{23} \\
        h_{31} & h_{32} & h_{33}
    \end{bmatrix}
    \cdot  
    \begin{bmatrix}
        x \\
        y \\
        1
    \end{bmatrix}
$$

**Свойства гомографии:**

- Гомография между изображениями если: 
    - плоская сцена снятая с разных ракурсов

    - трехмерная сцена при повороте камеры вокруг оптической оси

- В декартовых координатах гомография - нелинейное преобразование.

- При гомографии прямые линии отображаются в прямые. 
  При этом углы и масштаб меняются.

- Матрицу гомографии можно умножать на множитель - гомография не измениться.

- Восемь степеней свободы - для вычисления гомографии необходимо как минимум четыре пары соответствующих точек.

##### **Аффинные преобразования - частный случай гомографии**

<center>
    <figure>
        <img src="images/09/aff1.jpg" width=40%"/>
        <img src="images/09/aff2.jpg" width=30%"/>
    </figure>
</center>

- Если угол наклона плоскости сцены небольшой, то связь между
координатами точек можно аппроксимировать **аффинным преобразованием**:
$$
    \begin{bmatrix}
        x' \\ y' \\ 1
    \end{bmatrix}
    =   
    \begin{bmatrix}
        a_{11} & a_{12} & t_1 \\
        a_{21} & a_{22} & t_2 \\
        0 & 0 & 1
    \end{bmatrix}
    \begin{bmatrix}
        x \\ y \\ 1
    \end{bmatrix}
$$

- Аффинное преобразования является частным случаем гомографии:
$$
    \lambda=1
    \qquad
    h_{31} = h_{32} = 0 
    \qquad
    h_{33} = 1
$$

- Поскольку в третьей строке должна стоять единица, аффинное преобразование,
  в отличии от гомографии, при умножении на множитель измениться.

- В декартовых координатах - аффинное преобразование линейно:
$$
    \begin{bmatrix}
        x' \\ y'
    \end{bmatrix}
    =   
    \begin{bmatrix}
        a_{11} & a_{12} \\
        a_{21} & a_{22}
    \end{bmatrix}
    \begin{bmatrix}
        x \\ y
    \end{bmatrix}
    +
    \begin{bmatrix}
        t_1 \\ t_2
    \end{bmatrix}
$$
$$
    \mathbf{x}'=\mathbf{A}\cdot\mathbf{x} + \mathbf{t}
$$

- При аффинном преобразовании параллельные прямые остаются параллельными. Углы и масштаб меняются.

- В аффинном преобразовании шесть степеней свободы - необходимо как
  минимум три пары соответствующих точек, чтобы построить аффинное преобразование.

##### **Преобразование подобия - частный случай аффинного преобразования**

<center>
    <figure>
        <img src="images/09/satellite1.jpg" width=40%"/>
        <img src="images/09/satellite2.jpg" width=30%"/>
    </figure>
</center>

- Рассмотрим случай, когда сцена является плоской, и оптическая ось камеры перпендикулярна плоскости сцены.

- В этом случае преобразование между координатами точек на изображениях задается
  **преобразованием подобия**:
  $$
        \begin{bmatrix}
            x' \\ y' \\ 1
        \end{bmatrix}
        =
        \begin{bmatrix}
            \rho\cos\theta & -\rho\sin\theta & t_1 \\
            \rho\sin\theta & \rho\cos\theta & t_2 \\
            0 & 0 & 1
        \end{bmatrix}
        \begin{bmatrix}
            x \\ y \\ 1
        \end{bmatrix}
        \qquad
        \rho > 0
  $$
  Или в декартовых координатах:
  $$
    \begin{bmatrix}
            x' \\ y'
        \end{bmatrix}
        =
        \rho\cdot
        \begin{bmatrix}
            \cos\theta & -\sin\theta \\
            \sin\theta & \cos\theta
        \end{bmatrix}
        \begin{bmatrix}
            x \\ y
        \end{bmatrix} +
        \begin{bmatrix}
            t_1 \\ t_2
        \end{bmatrix}
  $$
  $$
    \mathbf{x}' = \rho\cdot\mathbf{R}\mathbf{x} + \mathbf{t}
  $$

- Преобразование подобия - частный случай аффинного преобразования.

- У преобразования подобия 4 параметра. Необходимо минимум 2 пары соответствующих точек, чтобы вычислить преобразование подобия.

<center>
    <figure>
        <img src="images/09/sim.png" width=30%"/>
    </figure>
</center>

- При преобразовании подобия углы между прямыми не меняются. Масштаб меняется в $\rho$ раз. 


#### **Вычисление аффинного преобразования**


- Аффинное преобразование содержит шесть параметров -
  четыре коэффициента матрицы $\mathbf{A}$ и два коэффициента вектора $\mathbf{t}$. Поэтому необходимо как минимум три соответствующих точки, чтобы вычисить параметры преобразование.
  $$
    \mathbf{x}_i 
    \quad\leftrightarrow\quad
    \mathbf{x}'_i
    \qquad\qquad
    i=1,2,\ldots,N\geq 3 
  $$

- Аффинное преобразование можно вычислить методом 
  наименьших квадратов, минимизирую среднюю ошибку проекции:
  $$
    \mathbf{\widehat A}\,,\mathbf{\widehat t} =
    \arg\min\limits_{\mathbf{A}\,,\mathbf{t}}
    \frac{1}{N}\sum_i
    \|\mathbf{x}'_i - (\mathbf{A}\cdot\mathbf{x}_i + \mathbf{t})\|^2
  $$

- Поскольку аффинное преобразование зависит линейно от параметров,
то существует глобальный минимум, который можно найти точно,
решив линейную систему уравнений:
    $$
        \mathbf{\Omega}\cdot\boldsymbol{\theta}=\boldsymbol{\beta}
    $$
    с неизвестным вектором
    $$
        \boldsymbol{\theta}=
        [a_{11}\,\, a_{12}\,\, t_1\,\, a_{21}\,\, a_{22}\,\, t_2]^\top
    $$
    где матрица $\mathbf{\Omega}$ размера $2N\times 6$ состоит из блоков:
    $$
        \begin{bmatrix}
            x_i & y_i & 1 & 0 & 0 & 0\\
            0 & 0 & 0 & x_i & y_i & 1
        \end{bmatrix}
    $$
    вектор $\boldsymbol{\beta}$ размера $2N\times 1$ состоит из блоков
    $$
        \begin{bmatrix}
            x'_i \\
            y'_i
        \end{bmatrix}
    $$

- Решение строиться с помощью псевдообратной матрицы $\boldsymbol\Omega^+$:
    $$
        \boldsymbol{\theta} = \boldsymbol\Omega^+
        \cdot\boldsymbol{\beta}\,,
        \qquad 
        \boldsymbol\Omega^+=
        \left(\boldsymbol\Omega^\top\boldsymbol\Omega\right)^{-1}\boldsymbol\Omega^\top
    $$

**Подготовим данные для задачи вычисления аффинного преобразования.**

In [ ]:
image1 = cv2.imread(DATA_DIR / "tiles2.png", cv2.IMREAD_COLOR_RGB)
image2 = cv2.imread(DATA_DIR / "tiles1.png", cv2.IMREAD_COLOR_RGB)

points1 = np.array([[161, 226],
    [315 , 232],
    [378, 310],
    [292, 384],
    [139, 378],
    [ 78 , 297]
], dtype=np.float32)

points2 = np.array([
    [110, 215], 
    [305, 215], 
    [400, 308], 
    [305, 405],
    [110, 405],
    [18, 308]
], dtype=np.float32)

out_image1 = image1.copy()
out_image2 = image2.copy()

for (x1, y1), (x2, y2) in zip(points1, points2):
    cv2.circle(out_image1, (int(x1), int(y1)), 7, (255, 0, 255), -1)
    cv2.circle(out_image2, (int(x2), int(y2)), 7, (255, 0, 255), -1)

mediapy.show_images({
        "Image 1": out_image1,
        "Image 2": out_image2,
    }, 
    border=True, width=400, columns=2
)

**Вычисление аффинного преобразования с помощью OpenCV.**

In [ ]:
# в OpenCV аффинное преобразование можно построить только по трем точкам
cv_A = cv2.getAffineTransform(points1[:3], points2[:3])

with np.printoptions(precision=4, suppress=True):
    print("OpenCV Affine Transform:\n", cv_A)

# преобразуем image1 с помощью аффинного преобразования
height, width = image2.shape[:2]
warped_image = cv2.warpAffine(image1, cv_A, (width, height), cv2.INTER_LINEAR)

mediapy.show_images({
        "Image 1": image1,
        "Image 2": image2,
        "Warped Image 1": warped_image,
    }, 
    border=True, width=400, columns=2
)

#### **Задача 3: Вычисление аффинного преобразования**

Напишите функцию, которая:

1) Вычисляет аффинное преобразование из image1 в image2.

2) Вычисляет среднюю ошибку проекции.

```python
def get_affine_transform(points1, points2, image1):
    """
    Вычисляет аффинное преобразование и среднюю ошибку проекции.
    Returns:
        A - матрица аффинного преобразования размером (2, 3)
        reproj_error - средняя ошибка проекции
    """
    ...
    return A, proj_error
```


Протестируем реализацию функции ``get_affine_transform()``.

In [ ]:
from homeworks.homework_09 import get_affine_transform

my_A, proj_error = get_affine_transform(points1, points2)

with np.printoptions(precision=4, suppress=True):
    print("Affine Transform:\n", my_A)

print(f"\nMean Projection Error: {proj_error:.4f} pixels")

# преобразуем image1 с помощью аффинного преобразования
height, width = image2.shape[:2]
warped_image = cv2.warpAffine(image1, my_A, (width, height), cv2.INTER_LINEAR)

mediapy.show_images({
        "Image 1": image1,
        "Image 2": image2,
        "Warped Image 1": warped_image,
    }, 
    border=True, width=400, columns=2
)

### **Домашнее задание 9**

#### **Теоретическая часть**

- Проективное преобразование изображений (гомография).

- Вычисление гомографии с помощью нелинейной оптимизации.

- Вычисление гомографии с помощью DLT алгоритма.

- Алгоритм RANSAC для оценки гомографии.

- Аффинное преобразование изображений, как частный случай гомографии.

- Преобразование подобия, как частный случай аффинного преобразования.

- Вычисление аффинного преобразования.
  
##### **Практическая часть**
Реализуйте в файле `homeworks/homework_09.py` функции из задач 1, 2 и 3.

Протестируйте их в ноутбуке `notebooks/notebook_09.ipynb`.